# Introduction

This notebook demonstrates the use of pre-trained MEGNet models to predict properties.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu

In [1]:
import torch
import dgl
import os
from pymatgen.core import Element, Structure, Lattice

## Import megnet related modules
from matgl.graph.converters import get_element_list, Pmg2Graph
from matgl.graph.compute import compute_pair_vector_and_distance
from matgl.models.megnet import MEGNet, MEGNetCalculator

# from matgl.utils.predictors import MEGNetCalculator
from matgl.layers.bond_expansion import BondExpansion

In [2]:
# Let's set the device first. Either "cuda" or "cpu"
device = torch.device("cpu")

# MP Formation energy 

The pre-trained model is traned on the Materaisl Project mp.2018.6.1.json dataset.

In [3]:
# load the pre-trained MEGNet model. By default it is the formation energy model.
model = MEGNet.load()
# map the model to CPU or GPU
model = model.to(device)
# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.57743067), ["Cu"], [[0, 0, 0]])
# create a graph converter
cry_graph = Pmg2Graph(element_types=model.element_types, cutoff=model.cutoff)
# convert pymatgen structure into dgl graph
graph, graph_attrs = cry_graph.get_graph_from_structure(structure=struct)
# define the Gaussian expansion
bond_expansion = BondExpansion(rbf_type="Gaussian", initial=0.0, final=5.0, num_centers=100, width=0.5)
# compute bond vectors and distances
bond_vec, bond_dist = compute_pair_vector_and_distance(graph)
# expand the bond distance into edge attributes through Gaussian expansion
graph.edata["edge_attr"] = bond_expansion(bond_dist)
# move all necessary inputs into device
graph = graph.to(device)
graph.edata["edge_attr"] = graph.edata["edge_attr"].to(device)
graph.ndata["node_type"] = graph.ndata["node_type"].to(device)
graph_attrs = torch.tensor(graph_attrs).to(device)
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, data_std=model.data_std, data_mean=model.data_mean)
Eform_pred = predictor(graph, graph_attrs)
print("The predicted formation energy for a FCC crystal is", float(Eform_pred.detach().numpy()), "eV/atom")


The predicted formation energy for a FCC crystal is -0.00748133659362793 eV/atom


/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  return th.as_tensor(data, dtype=dtype)
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/dgl/readout.py:443: DGLWarning: For a single graph, use a tensor of shape (1, *) for graph_feat. The support of shape (*) will be deprecated.
  dgl_warning(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped

# MP Band gap

The pre-trained model is traned on the Materails Project mp.2018.6.1.json dataset.

In [4]:
model = MEGNet.load("MP-2019.4.1-BandGap")
# map the model to CPU or GPU
model = model.to(device)
# read structure
struct = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.57743067), ["Cu"], [[0, 0, 0]])
# create a graph converter
cry_graph = Pmg2Graph(element_types=model.element_types, cutoff=model.cutoff)
# convert pymatgen structure into dgl graph
graph, graph_attrs = cry_graph.get_graph_from_structure(structure=struct)
# define graph label ("0": PBE, "1": GLLB-SC, "2": HSE, "3": SCAN) for multi-fidelity model
graph_attrs = torch.tensor([0])
# define the Gaussian expansion
bond_expansion = BondExpansion(rbf_type="Gaussian", initial=0.0, final=6.0, num_centers=100, width=0.5)
# compute bond vectors and distances
bond_vec, bond_dist = compute_pair_vector_and_distance(graph)
# expand the bond distance into edge attributes through Gaussian expansion
graph.edata["edge_attr"] = bond_expansion(bond_dist)
# move all necessary inputs into device
graph.ndata["node_type"].to(device)
model.data_mean.to(device)
model.data_std.to(device)
graph_attrs = torch.tensor(graph_attrs).to(device)
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, data_std=model.data_std, data_mean=model.data_mean)
BandGap = predictor(graph, graph_attrs)
print("The predicted PBE BandGap for a FCC Cu crystal is ", float(BandGap.detach().numpy()), "eV")

The predicted PBE BandGap for a FCC Cu crystal is  -0.0003764629364013672 eV


/var/folders/ql/m5k56v8n5sz5880n5sksmc9w0000gn/T/ipykernel_2773/534914385.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  graph_attrs = torch.tensor(graph_attrs).to(device)
